In [0]:
from keras.layers.core import Activation, Dense, Dropout, RepeatVector, SpatialDropout1D
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import GRU
from keras.layers.wrappers import TimeDistributed
from keras.models import Sequential, load_model
from keras.preprocessing import sequence
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
import collections
import nltk
import numpy as np
import os

Using TensorFlow backend.


In [0]:
model1 = load_model('model1.h5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [0]:
from keras.preprocessing.sequence import pad_sequences
import numpy as np
import nltk

nltk.download('treebank')
tagged_sentences = nltk.corpus.treebank.tagged_sents()
 
#print(tagged_sentences[0])
#print("Tagged sentences: ", len(tagged_sentences))
#print("Tagged words:", len(nltk.corpus.treebank.tagged_words()))
 
sentences, sentence_tags =[], [] 
for tagged_sentence in tagged_sentences:
    sentence, tags = zip(*tagged_sentence)
    sentences.append(np.array(sentence))
    sentence_tags.append(np.array(tags))
    
from sklearn.model_selection import train_test_split
 
 
(train_sentences, test_sentences, train_tags, test_tags) = train_test_split(sentences, sentence_tags, test_size=0.2)

def logits_to_tokens(sequences, index):
  token_sequences = []
  for categorical_sequence in sequences:
    token_sequence = []
    for categorical in categorical_sequence:
      token_sequence.append(index[np.argmax(categorical)])
    token_sequences.append(token_sequence)
  return token_sequence

test_samples = [
    "my name is John Patel, Kandarp Kakkad, Chirag Bajaj .".split(),
]

words, tags = set([]), set([])

for s in test_samples:
    for w in s:
        words.add(w.lower())

for ts in train_tags:
    for t in ts:
        tags.add(t)
        
        
word2index = {w: i + 2 for i, w in enumerate(list(words))}
word2index['-PAD-'] = 0  # The special value used for padding
word2index['-OOV-'] = 1  # The special value used for OOVs
tag2index = {t: i + 1 for i, t in enumerate(list(tags))}
tag2index['-PAD-'] = 0

for ts in train_tags:
    for t in ts:
        tags.add(t)

test_samples_X = []
for s in test_samples:
    s_int = []
    for w in s:
        try:
            s_int.append(word2index[w.lower()])
        except KeyError:
            s_int.append(word2index['-OOV-'])
    test_samples_X.append(s_int)
test_samples_X = pad_sequences(test_samples_X, maxlen=100, padding='post')
print(test_samples)
predictions = model1.predict(test_samples_X)
print(logits_to_tokens(predictions, {i: t for t, i in tag2index.items()}))

[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[['my', 'name', 'is', 'John', 'Patel,', 'Kandarp', 'Kakkad,', 'Chirag', 'Bajaj', '.']]


KeyError: ignored

In [0]:
test_samples = [
    "and but is John Patel, Kandarp Kakkad, Chirag Bajaj .".split(),
]
test_samples_X = []
for s in test_samples:
    s_int = []
    for w in s:
        try:
            s_int.append(word2index[w.lower()])
        except KeyError:
            s_int.append(word2index['-OOV-'])
    test_samples_X.append(s_int)
 
test_samples_X = pad_sequences(test_samples_X, maxlen=100, padding='post')
print(test_samples)
predictions = model1.predict(test_samples_X)
print(predictions)
print(predictions.argmax)
#print(logits_to_tokens(predictions, {i: t for t, i in tag2index.items()})

[['and', 'but', 'is', 'John', 'Patel,', 'Kandarp', 'Kakkad,', 'Chirag', 'Bajaj', '.']]
[[[3.0971918e-02 1.5803077e-04 1.7182190e-05 ... 2.3657225e-04
   1.3137695e-04 1.4381931e-04]
  [4.7612561e-05 1.1261850e-06 4.0319219e-07 ... 8.1519482e-07
   6.6128212e-07 6.3453359e-07]
  [7.5643446e-05 2.7651879e-05 3.0789677e-05 ... 2.1018252e-05
   1.6598637e-05 1.6886481e-05]
  ...
  [9.9999988e-01 1.4703766e-12 2.8214809e-15 ... 3.2447669e-12
   6.1056623e-13 1.3287751e-12]
  [9.9999940e-01 9.1841994e-12 4.1560312e-14 ... 2.1294603e-11
   4.4837883e-12 9.0597998e-12]
  [9.9998307e-01 3.7671102e-10 7.2925312e-12 ... 9.1399860e-10
   2.3326321e-10 4.0904938e-10]]]
<built-in method argmax of numpy.ndarray object at 0x7fd8214634e0>
